In [1]:
import backtrader as bt
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy

# 一、读取日度行情表

表内字段就是 Backtrader 默认情况下要求输入的 7 个字段： 'datetime' 、'open'、'high'、'low'、'close'、'volume'、'openinterest'，外加一个 'sec_code' 股票代码字段。

In [2]:
daily_price = pd.read_csv("./data/daily_price.csv", parse_dates=['datetime'])
daily_price

,datetime,sec_code,open,high,low,close,volume,openinterest
0,2019-01-02,600466.SH,33.064891,33.496709,31.954503,32.386321,10629352,0
1,2019-01-02,603228.SH,50.660230,51.458513,50.394136,51.120778,426147,0
2,2019-01-02,600315.SH,148.258423,150.480132,148.258423,149.558935,2138556,0
3,2019-01-02,000750.SZ,49.512579,53.154883,48.715825,51.561375,227557612,0
4,2019-01-02,002588.SZ,36.608672,36.608672,35.669988,35.763857,2841517,0
...,...,...,...,...,...,...,...,...
255967,2021-01-28,600717.SH,121.489201,122.011736,120.705400,120.966667,6022213,0
255968,2021-01-28,300558.SZ,134.155888,137.600704,130.700970,131.569750,5330301,0
255969,2021-01-28,600171.SH,39.774873,39.830040,38.864630,38.947380,12354183,0
255970,2021-01-28,600597.SH,47.190201,49.243025,46.250355,46.423484,32409940,0


In [3]:
# 筛选 600466.SH 和 603228.SH 2只股票的数据集
data1 = daily_price.query(f"sec_code=='600466.SH'").set_index('datetime').drop(columns=['sec_code'])
data2 = daily_price.query(f"sec_code=='603228.SH'").set_index('datetime').drop(columns=['sec_code'])
data2

,open,high,low,close,volume,openinterest
datetime,,,,,,
2019-01-02,50.660230,51.458513,50.394136,51.120778,426147,0
2019-01-03,50.609059,51.049137,50.107573,50.639762,492071,0
2019-01-04,50.199683,51.171950,49.278588,50.455543,665486,0
2019-01-07,50.864918,51.417575,50.353199,50.967262,689444,0
2019-01-08,51.110544,52.174920,50.250855,50.527183,931211,0
...,...,...,...,...,...,...
2021-01-22,59.955312,59.955312,57.478496,58.174461,6410959,0
2021-01-25,58.113052,58.133522,57.110044,57.498966,3445027,0
2021-01-26,57.498966,57.498966,54.305716,55.124498,7340180,0


# 一、 简单移动均线的测试案例

策略：5 日均线上穿 10 日均线 → 买入；5 日均线下穿 10 日均线 → 卖出

### 注：后面各交易操作都基于该案例进行测试

In [124]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=100) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close() # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已被处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.value,
                     order.executed.comm,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                        order.executed.value,
                        order.executed.comm,
                        order.data._name))
                
        
# 实例化大脑
cerebro1= bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(1000000000000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro1.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro1.addstrategy(TestStrategy)            
# 启动回测
result = cerebro1.run()  

2019-01-17, BUY EXECUTED, ref:3689，Price: 32.6331, Size: 100.00, Cost: 3263.3074, Comm 0.0000, Stock: 600466.SH
2019-01-29, SELL EXECUTED, ref:3690, Price: 33.9285, Size: -100.00, Cost: 3263.3074, Comm 0.0000, Stock: 600466.SH
2019-02-22, BUY EXECUTED, ref:3691，Price: 34.9155, Size: 100.00, Cost: 3491.5538, Comm 0.0000, Stock: 600466.SH
2019-02-27, SELL EXECUTED, ref:3692, Price: 37.5064, Size: -100.00, Cost: 3491.5538, Comm 0.0000, Stock: 600466.SH
2019-03-15, BUY EXECUTED, ref:3693，Price: 41.2077, Size: 100.00, Cost: 4120.7738, Comm 0.0000, Stock: 600466.SH
2019-03-18, SELL EXECUTED, ref:3694, Price: 44.1071, Size: -100.00, Cost: 4120.7738, Comm 0.0000, Stock: 600466.SH
2019-04-01, BUY EXECUTED, ref:3695，Price: 46.1428, Size: 100.00, Cost: 4614.2796, Comm 0.0000, Stock: 600466.SH
2019-04-03, SELL EXECUTED, ref:3696, Price: 47.0681, Size: -100.00, Cost: 4614.2796, Comm 0.0000, Stock: 600466.SH
2019-04-17, BUY EXECUTED, ref:3697，Price: 46.6363, Size: 100.00, Cost: 4663.6302, Comm 0.000

# 二、 打印资金和持仓

In [126]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        
        # 打印每日的资金和持仓情况
        print('date', self.data0.datetime.date(0))
        print('当前可用资金', self.broker.getcash())
        print('当前总资产', self.broker.getvalue())
        print('当前持仓量', self.broker.getposition(self.data).size)
        print('当前持仓成本', self.broker.getposition(self.data).price)
        
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=100) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close() # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已被处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.value,
                     order.executed.comm,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                        order.executed.value,
                        order.executed.comm,
                        order.data._name))
                
# 实例化大脑
cerebro1= bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(1000000000000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro1.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro1.addstrategy(TestStrategy)            
# 启动回测
result = cerebro1.run()  

date 2019-01-16
当前可用资金 1000000000000.0
当前总资产 1000000000000.0
当前持仓量 0
当前持仓成本 0.0
2019-01-17, BUY EXECUTED, ref:3811，Price: 32.6331, Size: 100.00, Cost: 3263.3074, Comm 0.0000, Stock: 600466.SH
date 2019-01-17
当前可用资金 999999996736.6926
当前总资产 999999999944.4806
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-18
当前可用资金 999999996736.6926
当前总资产 1000000000012.3376
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-21
当前可用资金 999999996736.6926
当前总资产 999999999981.4935
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-22
当前可用资金 999999996736.6926
当前总资产 999999999981.4935
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-23
当前可用资金 999999996736.6926
当前总资产 999999999987.6624
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-24
当前可用资金 999999996736.6926
当前总资产 999999999975.3247
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-25
当前可用资金 999999996736.6926
当前总资产 1000000000043.1818
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-28
当前可用资金 999999996736.6926
当前总资产 1000000000123.3765
当前持仓量 100
当前持仓成本 32.63307367
2019-01-29, SELL EXECUTED, ref:3812, Price: 33.9285, 

# 三、 滑点管理

In [128]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        
        # 打印每日的资金和持仓情况
        print('date', self.data0.datetime.date(0),
              'close', self.data0.close[0],
              'open', self.data0.open[0])
        print('当前可用资金', self.broker.getcash())
        print('当前总资产', self.broker.getvalue())
        print('当前持仓量', self.broker.getposition(self.data).size)
        print('当前持仓成本', self.broker.getposition(self.data).price)
        
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=100) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close() # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已被处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.value,
                     order.executed.comm,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                        order.executed.value,
                        order.executed.comm,
                        order.data._name))
                
# 实例化大脑
cerebro1= bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(1000000000000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro1.adddata(datafeed1, name='600466.SH')


# 设置百分比滑点 
cerebro1.broker.set_slippage_perc(perc=0.0001) 
# 设置固定滑点
# cerebro1.broker.set_slippage_fixed(fixed=0.001) 
# 有关滑点的其他设置
# cerebro1.broker.set_slippage_fixed(fixed=0.35, slip_open=False, slip_match=True, slip_out=False)
# cerebro1.broker.set_slippage_fixed(fixed=0.35, slip_open=True, slip_match=True, slip_out=False)
# cerebro1.broker.set_slippage_fixed(fixed=0.35, slip_open=True, slip_match=True, slip_out=True)
# cerebro1.broker.set_slippage_fixed(fixed=0.35, slip_open=True, slip_match=False, slip_out=True)

# 添加策略
cerebro1.addstrategy(TestStrategy)            
# 启动回测
result = cerebro1.run()  

date 2019-01-16 close 32.63307367 open 33.00320305
当前可用资金 1000000000000.0
当前总资产 1000000000000.0
当前持仓量 0
当前持仓成本 0.0
2019-01-17, BUY EXECUTED, ref:3933，Price: 32.6363, Size: 100.00, Cost: 3263.6337, Comm 0.0000, Stock: 600466.SH
date 2019-01-17 close 32.0778796 open 32.63307367
当前可用资金 999999996736.3663
当前总资产 999999999944.1543
当前持仓量 100
当前持仓成本 32.636336977367
date 2019-01-18 close 32.75645013 open 31.95450314
当前可用资金 999999996736.3663
当前总资产 1000000000012.0114
当前持仓量 100
当前持仓成本 32.636336977367
date 2019-01-21 close 32.44800898 open 33.06489128
当前可用资金 999999996736.3663
当前总资产 999999999981.1672
当前持仓量 100
当前持仓成本 32.636336977367
date 2019-01-22 close 32.44800898 open 32.38632075
当前可用资金 999999996736.3663
当前总资产 999999999981.1672
当前持仓量 100
当前持仓成本 32.636336977367
date 2019-01-23 close 32.50969721 open 32.44800898
当前可用资金 999999996736.3663
当前总资产 999999999987.336
当前持仓量 100
当前持仓成本 32.636336977367
date 2019-01-24 close 32.38632075 open 32.50969721
当前可用资金 999999996736.3663
当前总资产 999999999974.9984
当前持仓量 100

In [14]:
# 2019-01-17: 开盘价成交，实际成交价 为 开盘价*（1+0.0001）
32.63307367 * (1+0.0001)

32.636336977367

# 四、交易税费管理

以 考虑佣金和印花税的股票百分比费用 为例

In [130]:
# 考虑佣金和印花税的股票百分比费用
class StockCommission(bt.CommInfoBase):
    params = (('stamp_duty', 0.001),
              ('stocklike', True),  # 指定为期货模式
              ('commtype', bt.CommInfoBase.COMM_PERC),  # 使用百分比费用模式
              ('percabs', True),)  # commission 不以 % 为单位 # 印花税默认为 0.1%
    
    def _getcommission(self,size,price,pseudoexec):
        if size>0: # 买入时，只考虑佣金
            return abs(size) * price * self.p.commission
        elif size<0: # 卖出时，同时考虑佣金和印花税
            return abs(size) * price * (self.p.commission + self.p.stamp_duty)
        else:
            return 0
  

                
# 实例化大脑
cerebro1= bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(1000000000000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro1.adddata(datafeed1, name='600466.SH')

# 设置交易费用
comminfo = StockCommission(commission=0.0002, stamp_duty=0.001) # 实例化 
cerebro1.broker.addcommissioninfo(comminfo)

# 添加策略
cerebro1.addstrategy(TestStrategy)            
# 启动回测
result = cerebro1.run()  

date 2019-01-16 close 32.63307367 open 33.00320305
当前可用资金 1000000000000.0
当前总资产 1000000000000.0
当前持仓量 0
当前持仓成本 0.0
2019-01-17, BUY EXECUTED, ref:4055，Price: 32.6331, Size: 100.00, Cost: 3263.3074, Comm 0.6527, Stock: 600466.SH
date 2019-01-17 close 32.0778796 open 32.63307367
当前可用资金 999999996736.0399
当前总资产 999999999943.8279
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-18 close 32.75645013 open 31.95450314
当前可用资金 999999996736.0399
当前总资产 1000000000011.6849
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-21 close 32.44800898 open 33.06489128
当前可用资金 999999996736.0399
当前总资产 999999999980.8408
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-22 close 32.44800898 open 32.38632075
当前可用资金 999999996736.0399
当前总资产 999999999980.8408
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-23 close 32.50969721 open 32.44800898
当前可用资金 999999996736.0399
当前总资产 999999999987.0096
当前持仓量 100
当前持仓成本 32.63307367
date 2019-01-24 close 32.38632075 open 32.50969721
当前可用资金 999999996736.0399
当前总资产 999999999974.672
当前持仓量 100
当前持仓成本 32.63307367


In [27]:
date 2019-01-16 close 32.63307367 open 33.00320305
当前可用资金 1000000000000.0
当前总资产 1000000000000.0
当前持仓量 0
当前持仓成本 0.0
2019-01-17, BUY EXECUTED, ref:479，Price: 32.6331, Size: 100.00, Cost: 3263.3074, Comm 0.6527, Stock: 600466.SH
date 2019-01-17 close 32.0778796 open 32.63307367
当前可用资金 999999996736.0399
当前总资产 999999999943.8279
当前持仓量 100
当前持仓成本 32.63307367

In [39]:
# 2019-01-17 买入时只考虑 佣金： Cost * commission
100 * 32.6331 * 0.0002

0.6526620000000001

In [ ]:
2019-01-29, SELL EXECUTED, ref:488, Price: 33.9285, Size: -100.00, Cost: 3263.3074, Comm 4.0714, Stock: 600466.SH
date 2019-01-29 close 34.79216172 open 33.928526500000004
当前可用资金 1000000000124.8212
当前总资产 1000000000124.8212
当前持仓量 0
当前持仓成本 0.0

In [40]:
#  2019-01-29 卖出时 考虑佣金和印花税 
100 * 33.9285 * (0.0002+0.001)

4.071420000000001

# 五、成交量限制管理

In [50]:
# 将 原始数据的 volume 缩小 10000 倍，方便模拟结果
data1_ = data1.copy()
data1_.loc[:,'volume'] = (data1_.loc[:,'volume'] / 10000).astype('int')

In [51]:
data1_

,open,high,low,close,volume,openinterest
datetime,,,,,,
2019-01-02,33.064891,33.496709,31.954503,32.386321,1062,0
2019-01-03,32.262944,32.941515,31.399309,31.831127,860,0
2019-01-04,31.399309,33.558397,31.337621,33.496709,1276,0
2019-01-07,33.496709,34.360344,33.373332,33.620085,1058,0
2019-01-08,33.311644,34.113591,32.694762,33.743462,1001,0
...,...,...,...,...,...,...
2021-01-22,30.245430,30.312942,29.502796,29.772845,1718,0
2021-01-25,29.570309,29.570309,28.827675,28.962699,2364,0
2021-01-26,28.962699,29.232748,28.692651,28.760163,996,0


## bt.fillers.FixedSize(size)

In [100]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
         # 打印日志
       
        print('date', self.datas[0].datetime.date(0),
              'open', self.data0.open[0],
              'volume', self.data0.volume[0],
              '当前持仓量', self.broker.getposition(self.data).size,
              '当前持仓成本', self.broker.getposition(self.data).price)
    
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=2000) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close() # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Created, order.Submitted, order.Accepted, order.Canceled, order.Margin]:
            return
        # 已被处理的订单
        if order.status in [order.Partial, order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.remsize, # 剩余 数量
                     order.executed.value,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                         order.executed.remsize, # 剩余 数量
                        order.executed.value,
                        order.data._name))
                
# 实例化大脑
cerebro4 = bt.Cerebro()
# 设置初始资金
cerebro4.broker.set_cash(1000000)
# 设置成交量限制
cerebro4.broker.set_filler(bt.broker.fillers.FixedSize(size=3000))
# cerebro6.broker.set_filler(bt.broker.fillers.BarPointPerc(minmov=0.1, perc=0.5))
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1_, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro4.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro4.addstrategy(TestStrategy)            
# 启动回测
result = cerebro4.run()  

date 2019-01-16 open 33.00320305 volume 660.0 当前持仓量 0 当前持仓成本 0.0
2019-01-17, BUY EXECUTED, ref:2611，Price: 32.6331, Size: 869.00, Remsize: 1131.00, Cost: 28358.1410, Stock: 600466.SH
date 2019-01-17 open 32.63307367 volume 869.0 当前持仓量 869.0 当前持仓成本 32.63307367
date 2019-01-18 open 31.95450314 volume 890.0 当前持仓量 869.0 当前持仓成本 32.63307367
date 2019-01-21 open 33.06489128 volume 1292.0 当前持仓量 869.0 当前持仓成本 32.63307367
date 2019-01-22 open 32.38632075 volume 1030.0 当前持仓量 869.0 当前持仓成本 32.63307367
date 2019-01-23 open 32.44800898 volume 421.0 当前持仓量 869.0 当前持仓成本 32.63307367
date 2019-01-24 open 32.50969721 volume 615.0 当前持仓量 869.0 当前持仓成本 32.63307367
date 2019-01-25 open 32.94151482 volume 1732.0 当前持仓量 869.0 当前持仓成本 32.63307367
date 2019-01-28 open 33.311644199999996 volume 1686.0 当前持仓量 869.0 当前持仓成本 32.63307367
2019-01-29, SELL EXECUTED, ref:2612, Price: 33.9285, Size: -869.00, Remsize: 0.00, Cost: 28358.1410, Stock: 600466.SH
date 2019-01-29 open 33.928526500000004 volume 2256.0 当前持仓量 0.0 当前持仓成本 0

## bt.broker.fillers.FixedBarPerc(perc)

In [101]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
         # 打印日志
       
        print('date', self.datas[0].datetime.date(0),
              'open', self.data0.open[0],
              'volume', self.data0.volume[0],
              '当前持仓量', self.broker.getposition(self.data).size,
              '当前持仓成本', self.broker.getposition(self.data).price)
    
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=2000) # 以下一日开盘价买入2000股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close() # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Created, order.Submitted, order.Accepted, order.Canceled, order.Margin]:
            return
        # 已被处理的订单
        if order.status in [order.Partial, order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.remsize, # 剩余 数量
                     order.executed.value,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                         order.executed.remsize, # 剩余 数量
                        order.executed.value,
                        order.data._name))
                
# 实例化大脑
cerebro1 = bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(1000000)
# 设置成交量限制
cerebro1.broker.set_filler(bt.broker.fillers.FixedBarPerc(perc=50))  # 表示 50% 
# cerebro6.broker.set_filler(bt.broker.fillers.BarPointPerc(minmov=0.1, perc=0.5))
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1_, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro1.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro1.addstrategy(TestStrategy)            
# 启动回测
result = cerebro1.run()  

date 2019-01-16 open 33.00320305 volume 660.0 当前持仓量 0 当前持仓成本 0.0
2019-01-17, BUY EXECUTED, ref:2664，Price: 32.6331, Size: 434.00, Remsize: 1566.00, Cost: 14162.7540, Stock: 600466.SH
date 2019-01-17 open 32.63307367 volume 869.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-18 open 31.95450314 volume 890.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-21 open 33.06489128 volume 1292.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-22 open 32.38632075 volume 1030.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-23 open 32.44800898 volume 421.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-24 open 32.50969721 volume 615.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-25 open 32.94151482 volume 1732.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-28 open 33.311644199999996 volume 1686.0 当前持仓量 434.0 当前持仓成本 32.63307367
2019-01-29, SELL EXECUTED, ref:2665, Price: 33.9285, Size: -434.00, Remsize: 0.00, Cost: 14162.7540, Stock: 600466.SH
date 2019-01-29 open 33.928526500000004 volume 2256.0 当前持仓量 0.0 当前持仓成本 0

## bt.broker.fillers.BarPointPerc(minmov=0.01，perc=100.0)

In [99]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
         # 打印日志
       
        print('date', self.datas[0].datetime.date(0),
              'open', self.data0.open[0],
              'high', self.data0.high[0],
              'low', self.data0.low[0],
              'volume', self.data0.volume[0],
              '当前持仓量', self.broker.getposition(self.data).size,
              '当前持仓成本', self.broker.getposition(self.data).price)
    
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=1000) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close() # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Created, order.Submitted, order.Accepted, order.Canceled, order.Margin]:
            return
        # 已被处理的订单
        if order.status in [order.Partial, order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.remsize, # 剩余 数量
                     order.executed.value,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                        order.executed.size,
                        order.executed.remsize, # 剩余 数量
                        order.executed.value,
                        order.data._name))
                
# 实例化大脑
cerebro7 = bt.Cerebro()
# 设置初始资金
cerebro7.broker.set_cash(1000000)
# 设置成交量限制
cerebro7.broker.set_filler(bt.broker.fillers.BarPointPerc(minmov=0.1, perc=50))
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1_, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro7.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro7.addstrategy(TestStrategy)            
# 启动回测
result = cerebro7.run()  

date 2019-01-16 open 33.00320305 high 33.00320305 low 32.57138544 volume 660.0 当前持仓量 0 当前持仓成本 0.0
2019-01-17, BUY EXECUTED, ref:2560，Price: 32.6331, Size: 36.00, Remsize: 964.00, Cost: 1174.7907, Stock: 600466.SH
date 2019-01-17 open 32.63307367 high 32.94151482 low 31.83112668 volume 869.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-18 open 31.95450314 high 32.81813836 low 31.95450314 volume 890.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-21 open 33.06489128 high 33.06489128 low 32.0778796 volume 1292.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-22 open 32.38632075 high 32.6947619 low 32.01619137 volume 1030.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-23 open 32.44800898 high 32.57138544 low 32.0778796 volume 421.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-24 open 32.50969721 high 32.57138544 low 31.89281491 volume 615.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-25 open 32.94151482 high 34.05190296 low 32.75645013 volume 1732.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-28 open 33.

In [82]:
parts = (32.94151482 - 31.83112668 + 0.1)  // 0.1 
parts

12.0

In [86]:
(869.0 / parts) * 50 / 100

36.208333333333336

# 交易时机的管理

## 正常模式：当日收盘后下单，次日以开盘价成交

In [120]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:  
                print('{} Send Buy, open {}'.format(self.data.datetime.date(),self.data.open[0]))
                self.order = self.buy(size=100) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:   
            print('{} Send Sell, open {}'.format(self.data.datetime.date(),self.data.open[0]))
            self.order = self.close() # 平仓，以下一日开盘价卖出
    
    def notify_order(self, order):
        if order.status != order.Completed:
            return
        self.order = None
        print('{} {} Executed at price {}'.format(
            bt.num2date(order.executed.dt).date(),
            'Buy' * order.isbuy() or 'Sell', order.executed.price))        

    
# 实例化大脑
cerebro5= bt.Cerebro(cheat_on_open=False)
# 设置初始资金
cerebro5.broker.set_cash(1000000000000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro5.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro5.addstrategy(TestStrategy)            
# 启动回测
result = cerebro5.run()  

2019-01-16 Send Buy, open 33.00320305
2019-01-17 Buy Executed at price 32.63307367
2019-01-28 Send Sell, open 33.311644199999996
2019-01-29 Sell Executed at price 33.928526500000004
2019-02-21 Send Buy, open 35.47073225
2019-02-22 Buy Executed at price 34.91553818
2019-02-26 Send Sell, open 37.07462623
2019-02-27 Sell Executed at price 37.50644384
2019-03-14 Send Buy, open 41.82461994
2019-03-15 Buy Executed at price 41.20773764
2019-03-15 Send Sell, open 41.20773764
2019-03-18 Sell Executed at price 44.10708445
2019-03-29 Send Buy, open 43.55189038
2019-04-01 Buy Executed at price 46.14279604
2019-04-02 Send Sell, open 48.1168194
2019-04-03 Sell Executed at price 47.06811949
2019-04-16 Send Buy, open 47.56162533
2019-04-17 Buy Executed at price 46.63630188
2019-05-17 Send Sell, open 40.93009102
2019-05-20 Sell Executed at price 39.77351074
2019-05-20 Send Buy, open 39.77351074
2019-05-21 Buy Executed at price 39.25947506
2019-06-12 Send Sell, open 39.90201966
2019-06-13 Sell Executed 

## cheat_on_open

In [121]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next_open(self):
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:  
                print('{} Send Buy, open {}'.format(self.data.datetime.date(),self.data.open[0]))
                self.order = self.buy(size=100) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:  
            print('{} Send Sell, open {}'.format(self.data.datetime.date(),self.data.open[0]))
            self.order = self.close() # 平仓，以下一日开盘价卖出
    
    def notify_order(self, order):
        if order.status != order.Completed:
            return
        self.order = None
        print('{} {} Executed at price {}'.format(
            bt.num2date(order.executed.dt).date(),
            'Buy' * order.isbuy() or 'Sell', order.executed.price))        

    
# 实例化大脑
cerebro5= bt.Cerebro(cheat_on_open=True)
# 设置初始资金
cerebro5.broker.set_cash(1000000000000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro5.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro5.addstrategy(TestStrategy)            
# 启动回测
result = cerebro5.run()  

2019-01-17 Send Buy, open 32.63307367
2019-01-17 Buy Executed at price 32.63307367
2019-01-29 Send Sell, open 33.928526500000004
2019-01-29 Sell Executed at price 33.928526500000004
2019-02-22 Send Buy, open 34.91553818
2019-02-22 Buy Executed at price 34.91553818
2019-02-27 Send Sell, open 37.50644384
2019-02-27 Sell Executed at price 37.50644384
2019-03-15 Send Buy, open 41.20773764
2019-03-15 Buy Executed at price 41.20773764
2019-03-18 Send Sell, open 44.10708445
2019-03-18 Sell Executed at price 44.10708445
2019-04-01 Send Buy, open 46.14279604
2019-04-01 Buy Executed at price 46.14279604
2019-04-03 Send Sell, open 47.06811949
2019-04-03 Sell Executed at price 47.06811949
2019-04-17 Send Buy, open 46.63630188
2019-04-17 Buy Executed at price 46.63630188
2019-05-20 Send Sell, open 39.77351074
2019-05-20 Sell Executed at price 39.77351074
2019-05-21 Send Buy, open 39.25947506
2019-05-21 Buy Executed at price 39.25947506
2019-06-13 Send Sell, open 40.09478304
2019-06-13 Sell Executed

## Cheat-On-Close

In [122]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:  
                print('{} Send Buy, close {}'.format(self.data.datetime.date(),self.data.close[0]))
                self.order = self.buy(size=100) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:
            print('{} Send Sell, close {}'.format(self.data.datetime.date(),self.data.close[0]))
            self.order = self.close() # 平仓，以下一日开盘价卖出
    
    def notify_order(self, order):
        if order.status != order.Completed:
            return
        self.order = None
        print('{} {} Executed at price {}'.format(
            bt.num2date(order.executed.dt).date(),
            'Buy' * order.isbuy() or 'Sell', order.executed.price))        

    
# 实例化大脑
cerebro5= bt.Cerebro()
# 设置初始资金
cerebro5.broker.set_cash(1000000000000)
# 当日下单，当日收盘价成交
cerebro5.broker.set_coc(True)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro5.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro5.addstrategy(TestStrategy)            
# 启动回测
result = cerebro5.run()  

2019-01-16 Send Buy, close 32.63307367
2019-01-16 Buy Executed at price 32.63307367
2019-01-28 Send Sell, close 33.86683827
2019-01-28 Sell Executed at price 33.86683827
2019-02-21 Send Buy, close 34.85384995
2019-02-21 Buy Executed at price 34.85384995
2019-02-26 Send Sell, close 37.75319676
2019-02-26 Sell Executed at price 37.75319676
2019-03-14 Send Buy, close 41.20773764
2019-03-14 Buy Executed at price 41.20773764
2019-03-15 Send Sell, close 42.62656693
2019-03-15 Sell Executed at price 42.626566929999996
2019-03-29 Send Buy, close 45.52591374
2019-03-29 Buy Executed at price 45.52591373999999
2019-04-02 Send Sell, close 47.43824887
2019-04-02 Sell Executed at price 47.43824887000001
2019-04-16 Send Buy, close 46.63630188
2019-04-16 Buy Executed at price 46.63630188
2019-05-17 Send Sell, close 39.96627412
2019-05-17 Sell Executed at price 39.96627412
2019-05-20 Send Buy, close 39.38798398
2019-05-20 Buy Executed at price 39.38798398
2019-06-12 Send Sell, close 39.96627412
2019-06